In [9]:
import pandas as pd

# Load the CSV into a DataFrame
stock_data = pd.read_csv(r"C:\Users\Sezy\OneDrive\Personal_Projects\Trading_Algorithm\all_stocks_historical_data.csv")

# Print the first few rows to ensure it's loaded correctly
print(stock_data.head())

                        Date        Open        High         Low       Close  \
0  2020-02-04 00:00:00-05:00  169.431858  172.779557  168.637973  172.282181   
1  2020-02-05 00:00:00-05:00  176.022006  176.184606  170.646562  172.071716   
2  2020-02-06 00:00:00-05:00  173.095165  175.821154  172.224760  175.639420   
3  2020-02-07 00:00:00-05:00  174.893354  177.552382  174.539444  175.888092   
4  2020-02-10 00:00:00-05:00  175.591599  180.622707  175.275957  180.488800   

     Volume  Dividends  Stock Splits  daily_returns  annualized_volatility  \
0  36433300        0.0           0.0            NaN                0.30621   
1  39186300        0.0           0.0      -0.001222                0.30621   
2  27751400        0.0           0.0       0.020734                0.30621   
3  33529100        0.0           0.0       0.001416                0.30621   
4  35844300        0.0           0.0       0.026157                0.30621   

  ticker  Capital Gains  
0   MSFT            NaN 

In [10]:
# Standardize column names to lowercase and strip any extra spaces
stock_data.columns = stock_data.columns.str.lower().str.strip()

# Check the columns to confirm they have been standardized
print(stock_data.columns)

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'dividends',
       'stock splits', 'daily_returns', 'annualized_volatility', 'ticker',
       'capital gains'],
      dtype='object')


In [14]:
# Group by ticker and calculate the SMA for each stock separately
stock_data['sma_50'] = stock_data.groupby('ticker')['close'].rolling(window=50, min_periods=1).mean().reset_index(level=0, drop=True)
stock_data['sma_200'] = stock_data.groupby('ticker')['close'].rolling(window=200, min_periods=1).mean().reset_index(level=0, drop=True)

# Display the first few rows to check if the columns are added correctly
print(stock_data[['date', 'ticker', 'close', 'sma_50', 'sma_200']].head())

                        date ticker       close      sma_50     sma_200
0  2020-02-04 00:00:00-05:00   MSFT  172.282181  172.282181  172.282181
1  2020-02-05 00:00:00-05:00   MSFT  172.071716  172.176949  172.176949
2  2020-02-06 00:00:00-05:00   MSFT  175.639420  173.331106  173.331106
3  2020-02-07 00:00:00-05:00   MSFT  175.888092  173.970352  173.970352
4  2020-02-10 00:00:00-05:00   MSFT  180.488800  175.274042  175.274042


In [16]:
stock_data.to_csv(r"C:\Users\Sezy\OneDrive\Personal_Projects\Trading_Algorithm\modified_stock_data.csv", index=False)

# Confirm that the data has been saved
print("Modified data saved successfully!")

Modified data saved successfully!


In [6]:
print(stock_data.columns)

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'dividends',
       'stock splits', 'daily_returns', 'annualized_volatility', 'ticker',
       'capital gains'],
      dtype='object')


In [17]:
import numpy as np

# 1. Calculate the RSI (14-day period is typical)
delta = stock_data['close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
rsi = 100 - (100 / (1 + rs))

# Add RSI to the DataFrame
stock_data['rsi'] = rsi

# 2. Calculate the MACD (12-day and 26-day EMAs)
ema_12 = stock_data['close'].ewm(span=12, adjust=False).mean()
ema_26 = stock_data['close'].ewm(span=26, adjust=False).mean()

# MACD = 12-day EMA - 26-day EMA
macd = ema_12 - ema_26

# Signal line = 9-day EMA of the MACD
signal_line = macd.ewm(span=9, adjust=False).mean()

# Add MACD and Signal line to the DataFrame
stock_data['macd'] = macd
stock_data['macd_signal'] = signal_line

# Display the first few rows to check the new columns
print(stock_data[['date', 'ticker', 'close', 'rsi', 'macd', 'macd_signal']].head())

                        date ticker       close  rsi      macd  macd_signal
0  2020-02-04 00:00:00-05:00   MSFT  172.282181  NaN  0.000000     0.000000
1  2020-02-05 00:00:00-05:00   MSFT  172.071716  NaN -0.016789    -0.003358
2  2020-02-06 00:00:00-05:00   MSFT  175.639420  NaN  0.254851     0.048284
3  2020-02-07 00:00:00-05:00   MSFT  175.888092  NaN  0.484608     0.135549
4  2020-02-10 00:00:00-05:00   MSFT  180.488800  NaN  1.026102     0.313659


In [18]:
# Drop rows with NaN values in any of the columns
stock_data_cleaned = stock_data.dropna()

# Display the first few rows of the cleaned data
print(stock_data_cleaned[['date', 'ticker', 'close', 'rsi', 'macd', 'macd_signal']].head())

                           date ticker       close        rsi       macd  \
8807  2020-02-05 00:00:00-05:00    SPY  308.602631  83.764968  21.331519   
8808  2020-02-06 00:00:00-05:00    SPY  309.640991  83.568337  30.245125   
8809  2020-02-07 00:00:00-05:00    SPY  307.990753  82.825374  36.752404   
8810  2020-02-10 00:00:00-05:00    SPY  310.289948  82.555381  41.615279   
8811  2020-02-11 00:00:00-05:00    SPY  310.827698  82.582323  44.993874   

      macd_signal  
8807     4.167504  
8808     9.383028  
8809    14.856903  
8810    20.208579  
8811    25.165638  


In [20]:
# Forward fill NaN values
stock_data_filled = stock_data.ffill()

# Display the first few rows
print(stock_data_filled[['date', 'ticker', 'close', 'rsi', 'macd', 'macd_signal']].head())

                        date ticker       close  rsi      macd  macd_signal
0  2020-02-04 00:00:00-05:00   MSFT  172.282181  NaN  0.000000     0.000000
1  2020-02-05 00:00:00-05:00   MSFT  172.071716  NaN -0.016789    -0.003358
2  2020-02-06 00:00:00-05:00   MSFT  175.639420  NaN  0.254851     0.048284
3  2020-02-07 00:00:00-05:00   MSFT  175.888092  NaN  0.484608     0.135549
4  2020-02-10 00:00:00-05:00   MSFT  180.488800  NaN  1.026102     0.313659


In [21]:
# Save the cleaned and enhanced DataFrame to a new CSV file
stock_data_filled.to_csv('cleaned_stock_data.csv', index=False)

In [ ]:
import pandas as pd

# Load the options data (assuming you have this in a DataFrame)
options_data = pd.read_csv("options_data.csv")  # Replace with your actual file

# Function to generate 10 ITM strike prices
def generate_itm_strikes(current_price):
    if current_price >= 200:
        increment = 5
    else:
        increment = 1
    return [current_price - (i * increment) for i in range(1, 11)]  # 10 ITM strikes

# Apply function to generate ITM strike prices for each row
options_data['ITM_Strikes'] = options_data['Current Price'].apply(generate_itm_strikes)

# Explode the list into separate rows (so each strike has its own row)
itm_strikes_df = options_data.explode('ITM_Strikes')

# Rename column for clarity
itm_strikes_df = itm_strikes_df.rename(columns={'ITM_Strikes': 'Strike Price'})

# Display sample output
print(itm_strikes_df.head(20))

In [24]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Load the options data
options_data = pd.read_csv("options_data.csv")

# Clean column names by stripping whitespace
options_data.columns = options_data.columns.str.strip()

# Calculate moneyness: (Strike Price - Current Price) / Current Price
options_data['moneyness'] = (options_data['Strike Price'] - options_data['Current Price']) / options_data['Current Price']

# Define features and target
X = options_data[['Current Price', 'Strike Price', 'moneyness']]
y = options_data['Last Price']  # Option premium

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X, y)

# Predict on the training data and evaluate the model
y_pred = model.predict(X)
mae = mean_absolute_error(y, y_pred)
print("Mean Absolute Error on training data:", mae)

Mean Absolute Error on training data: 3.087767840584091


In [25]:
def predict_premium(row, strike_col):
    current_price = row['open']
    strike = row[strike_col]
    # Calculate moneyness: how far the strike is from the current price, relative to the current price.
    moneyness = (strike - current_price) / current_price
    # Create the feature vector as expected by the model: [Current Price, Strike Price, Moneyness]
    features = [[current_price, strike, moneyness]]
    # Use the trained model to predict the premium.
    return model.predict(features)[0]

# Apply the prediction for each ITM strike column and store the result in a new column.
for col in ['ITM1', 'ITM2', 'ITM3', 'ITM4', 'ITM5']:
    merged_data[f'Estimated_Premium_{col}'] = merged_data.apply(lambda row: predict_premium(row, col), axis=1)

# Display a sample of the results.
print(merged_data[['date', 'open', 'ITM1', 'Estimated_Premium_ITM1']].head())

KeyError: 'ITM1'